> [*JaberAlJ*](https://github.com/JaberAlJ)

# Uninformed Search Techniques

## Contents  <a class="anchor" id="top"></a>
* [1 Objectives](#objectives)
* [2 Overview](#overview)
* [3 Setting Up](#settingup)
* [4 Code for the 8-puzzle Problem](#puzzle)
* [5 Code for the Path Finding Problem](#path)

## 1 Objectives <a class="anchor" id="objectives"></a>

* To identify the building blocks of uninformed search algorithms and implement them.
* To apply uninformed search techniques to puzzle solving and path-finding problems.
* To differentiate between breadth-first (BFS), depth-first (DFS), depth-limited (DLS), and depth first iterative deepening (DFID) search algorithms.
* To compare the performance of the above search algorithms when used to solve simple problems.

### 1.1 Required Submission and Marking Scheme

This lab has a total of $10$ marks and worth $5\%$ of the course total. You should complete the following activities, each is worth $2$ marks:

* [Activity 5](#act5): Implementation of the *BFS* search algorithm for solving the 8-puzzle problem.
* [Activity 6](#act6): Implementation of the *DFID* search algorithm for solving the 8-puzzle problem.
* [Activity 7](#act7): Questions based on the implementations of `BFSSlover`,  `DFSSlover` and `DFIDSlover`.
* [Activity 9](#act9): Write the `DFIDPath` client code.
* [Activity 10](#act10): Fix the code of `dfs_path.py` and write the client code for `DFSPath` class.

## 2 Overview  <a class="anchor" id="overview"></a>

One of the fundamental techniques in artificial intelligence is solving problems through search. A search problem is represented as a _state space graph_ or a _search tree_. In either representation, nodes correspond to problem situations, and a given problem is reduced to finding a path between a _start_ node and a _goal_ node. In this lab, we look into how to use uninformed search algorithms to solve two problems: the _N-puzzle_ problem and _finding a path_ between two vertices in an _undirected_ graph. 

## 3 Setting Up <a class="anchor" id="settingup"></a>

To complete this lab, you will familiarize yourself with the structure of the code associated with this notebook. 

#### Activity 1 <a class="anchor" id="act1"></a> &nbsp; [Top](#top)

Based on the structure of the files in this Lab answer each of the following questions.

* Study the the structure of the files associated with the notebook. Fill the list `packes` below with the names of modules used to organize the code.

* What are the sub-folders under the `data` folder? Fill in the list `data` with the names of these sub-folders.

In [1]:
### START CODE HERE  (Activity 1) ###

# Fill the list below with the names of packages used to organize the code. (list of strings)
packages = ['search.graph', 'search.puzzle', 'stdlib']          

# Fill the list below with the names of the sub-folders in the data folder. (list of strings)
data   = ['graph', 'puzzle'] 

### END CODE HERE ###

## 4 Code for the 8-puzzle Problem <a class="anchor" id="puzzle"></a>

The 8-puzzle problem is a puzzle invented and popularized by Noyes Palmer Chapman in the 1870s. It is played on a 3-by-3 grid with 8 square blocks labelled 1 through 8 and a blank square. Your goal is to rearrange the blocks so that they are in order. You are permitted to slide blocks horizontally or vertically into the blank square. The following shows a sequence of legal moves from an initial board position (left) to the goal position (right). 
```
   1  3        1     3        1  2  3        1  2  3        1  2  3
4  2  5   =>   4  2  5   =>   4     5   =>   4  5      =>   4  5  6
7  8  6        7  8  6        7  8  6        7  8  6        7  8 

initial                                                      goal
```

In the following sections, we present an implementation of the _depth-first_ search algorithm to solve this problem. In the following activities, you shall implement the __breadth-first__ and __iterative deepening__ algorithms that solve this problem. Last, you will compare the performance of the different search algorithms on solving the 8-puzzle problem. Note that you are provided with both _depth-first_ and _depth-limited_ search algorithms. 

### 4.1 `DFSSolver`, A class that solves the 8-puzzle problem

In this section we present the DFS implementation for solving the 8-puzzle problem. The class that contains the implementation is found in `search/puzzle/dfs_solver.py`. In this class a method named `__dfs()` implements the _depth-first search_ algorithm for solving the 8-puzzle problem. Note that we only show the relevant method that contains the code for the algorithm. The remaining code is found in `dfs_solver.py`.

```
 1:      def __dfs(self, initial:Board):
 2:          """
 3:          Run Depth-First Search on the initial board
 4:          """
 5:          agenda = Stack()
 6:          visited = []
 7:  
 8:          # create a new node and add it to the agenda
 9:          firstNode = Node(initial, 0, 0, None)
10:          agenda.push(firstNode)
11:  
12:          # DFS loop
13:          while not(agenda.isEmpty()):
14:              node:Node = agenda.pop()
15:  
16:              # if the board is not visited 
17:              if not(node.state in visited):
18:  
19:                  # add the state to visited
20:                  visited.append(node.state)
21:  
22:                  if node.state.is_goal():
23:                      # found the goal
24:                      self._goal = node
25:                      self._found = True
26:                      break
27:  
28:                  # put the successor nodes in the agenda
29:                  for successor in node.state.neighbors():
30:  
31:                      # create a succesor node and add it to agenda
32:                      # for time being the cost to a node from a parent is 1
33:                      neigborN = Node(successor, node.depth + 1, node.cost + 1, node)
34:  
35:                      agenda.push(neigborN)
```

#### 4.1.1 What do you see for the first time in `__dfs()`

*  In Line $1$ we create a method named `__dfs()` which takes a single parameter, specifically an _initial_
	unsolved puzzle board.
*  In Line $5$ and $6$ we create the two data structures that are needed to keep track of the _search tree_. The `agenda` is a `stack` of `nodes` in search-tree. A `Node` class has four member variables: a _state_ configuration (i.e., a board), the _depth_ of node in the search tree, the _cost_ to get to the node in the tree, and a pointer the _parent_ node. The `Node` class is defined in the same package as the `DFSSolver` class. The `visited` object is an instance of python list. We use this object to maintain a list of nodes that has been encountered by the search algorithm.

* Lines $12$ - $35$ contains the main loop of the DFS algorithm. We iterate over the agenda as long as it is not empty. Note that if we find a solution to a puzzle we exit this loop, after we save the goal node and set `found` to `True`, hence the `break` statement in line $26$.

* Lines $29$ to $35$ implement the logic of adding successor nodes into the \code{agenda}. We iterate over the neighbors of a board using the `neighbors()` method defined by the `Board` class. The property `node.state` in line $29$ returns an object of type `Board` which is to call the `neighbors()`. The method `neighbors()`, apply the possible moves to a board (i.e., _left_, _up_, _right_ or _down_) and return new board configuration based on the move that been made.

### 4.2 Activites on the 8-Puzzle Problem

#### Activity 2 <a class="anchor" id="act2"></a> &nbsp; [Top](#top)

Having completed Activity 1, explore that code and input files that has been provided for solving the 8-puzzle problem. Answer or note each of the following.

* What is the name of the package that contains the 8-puzzle code? Fill in the variable name below.
* What are the code files that contain the **complete** implementation for search algorithms? Fill in the list named `complete_code`

In [2]:
### START CODE HERE  (Activity 2) ###

# Fill the list below with the names of package that contains the 8-puzzle code  (str)
puzzle_package = 'search.puzzle'          

# Fill in list below with the names of the files that contain complete implementation (list of strings)
complete_code   = ['dfs_solver.py', 'dls_solver.py']

### END CODE HERE ###

#### The `DFSSolver` Client Code

Study the code provided to you in `dfs_solver.py` and `dls_solver.py`. Below you will find the client code for using the classes `DFSSolver` and `DLSSolver`. Run each of the code cells below and answer the questions that follow.

Next, study the code below which shows how to use the `DFSSolver` to solve a given puzzle. 

In [3]:
from search.puzzle import Board      # used for the board configuration
from stdlib import StopwatchCPU      # a utility to compute the execution time in second
from search.puzzle import DFSSolver  # the solver class

# create a board by reading from a file
board = Board(filename="data/puzzle/puzzle2x2-01.txt")

# create timer object
timer = StopwatchCPU()

# create a solver
solver = DFSSolver(board)

# print how long it took to solve the puzzle
print("DFS finished in {0:.2f} seconds".format(timer.elapsedTime()))

# if we found the goal
if solver.found:
    print(f"Found the solution after {solver.moves} moves")
    solver.print_solution() # uncomment this line to see the solution
else:
    print("No solution found")

DFS finished in 0.00 seconds
Found the solution after 1 moves
Number of moves = 1
 1  2 
 0  3 

 1  2 
 3  0 



#### Activity 3 <a class="anchor" id="act3"></a> &nbsp; [Top](#top)

Run the `DFSSolver` on `puzzle2x2-01.txt`, `puzzle3x3-01.txt`, and `puzzle3x3-02.txt`. How long it takes solve each board in seconds? What is the number of moves for each case?

In [4]:
### START CODE HERE  (Activity 3) ###

# Fill in the following variables with time (in seconds) for solving each of the boards. (float)
dfs_time_puzzle_2_2_1 = 0.00          # time for puzzle2x2-01.txt
dfs_time_puzzle_3_3_1 = 989.44        # time for puzzle3x3-01.txt
dfs_time_puzzle_3_3_2 = 2560.66       # time for puzzle3x3-02.txt

# Fill in the number of moves taking to solve each of the boards, (int)
dfs_moves_puzzle_2_2_1 = 1           # moves for puzzle2x2-01.txt
dfs_moves_puzzle_3_3_1 = 58574       # moves for puzzle3x3-01.txt
dfs_moves_puzzle_3_3_2 = 90195       # moves for puzzle3x3-02.txt

### END CODE HERE ###

#### The `DLSSolver` Client Code

Study the client code for solving the 8-puzzle problem using the *depth_limited_search* approached. Note that the limit is passed as a parameter to the constructor of the `DLSSolver` class.

In [5]:
from search.puzzle import Board      # used for the board configuration
from stdlib import StopwatchCPU      # a utility to compute the execution time in second
from search.puzzle import DLSSolver  # the solver class

# create a board by reading from a file
board = Board(filename="data/puzzle/puzzle3x3-02.txt")

# create timer object
timer = StopwatchCPU()

# create a solver
solver = DLSSolver(board, limit=13)

# print how long it took to solve the puzzle
print("DLS finished in {0:.2f} seconds".format(timer.elapsedTime()))

# if we found the goal
if solver.found:
    print(f"Found the solution after {solver.moves} moves")
    solver.print_solution() # uncomment this line to see the solution
else:
    print("No solution found")

DLS finished in 0.16 seconds
Found the solution after 11 moves
Depth = 13
Number of moves = 11
 1  0  2 
 7  5  4 
 8  6  3 

 1  5  2 
 7  0  4 
 8  6  3 

 1  5  2 
 7  4  0 
 8  6  3 

 1  5  2 
 7  4  3 
 8  6  0 

 1  5  2 
 7  4  3 
 8  0  6 

 1  5  2 
 7  4  3 
 0  8  6 

 1  5  2 
 0  4  3 
 7  8  6 

 1  5  2 
 4  0  3 
 7  8  6 

 1  0  2 
 4  5  3 
 7  8  6 

 1  2  0 
 4  5  3 
 7  8  6 

 1  2  3 
 4  5  0 
 7  8  6 

 1  2  3 
 4  5  6 
 7  8  0 



#### Activity 4 <a class="anchor" id="act4"></a> &nbsp; [Top](#top)

Run the `DLSSolver` on `puzzle2x2-01.txt`, `puzzle3x3-01.txt`, and `puzzle3x3-02.txt`. How long it takes solve each board in seconds? What is the number of moves for each case? At what depth does DLS find a solution to each one of these boards?

In [6]:
### START CODE HERE  (Activity 4) ###

# Fill in the following variables with time (in seconds) for solving each of the boards. (float)
dls_time_puzzle_2_2_1 = 0.00       # time for puzzle2x2-01.txt
dls_time_puzzle_3_3_1 = 0.01       # time for puzzle3x3-01.txt
dls_time_puzzle_3_3_2 = 0.17       # time for puzzle3x3-02.txt

# Fill in the number of moves taking to solve each of the boards, (int)
dls_moves_puzzle_2_2_1 = 1        # moves for puzzle2x2-01.txt
dls_moves_puzzle_3_3_1 = 8        # moves for puzzle3x3-01.txt
dls_moves_puzzle_3_3_2 = 11       # moves for puzzle3x3-02.txt

# Fill in the following variables depth found by DLS to solve the boards. (int)
dls_depth_puzzle_2_2_1 = 1        # depth for puzzle2x2-01.txt
dls_depth_puzzle_3_3_1 = 8        # depth for puzzle3x3-01.txt
dls_depth_puzzle_3_3_2 = 13       # depth for puzzle3x3-02.txt

### END CODE HERE ###

#### Activity 5 <a class="anchor" id="act5"></a> &nbsp; [Top](#top)

Using the implementation of `__dfs()` as a template, complete the code in `bfs_solver.py` by implementing the `__bfs()` method. The method should encode the breadth-first search algorithm to solve the 8-puzzle problem. Ensure that your code follows the algorithm we studied in the class.

Remember to fill in the following comments at the top of the source code file.

```Python
# FileName    : bfs_solver.py
# Student ID  : <YOUR ID>
# Student Name: <YOUR NAME>
```

#### The `BFSSolver` Client Code

Client code for solving the 8-puzzle problem using the *Breadth_First_Search* approached.

In [7]:
from search.puzzle import Board      # used for the board configuration
from stdlib import StopwatchCPU      # a utility to compute the execution time in second
from search.puzzle import BFSSolver  # the solver class

# create a board by reading from a file
board = Board(filename="data/puzzle/puzzle3x3-02.txt")

# create timer object
timer = StopwatchCPU()

# create a solver
solver = BFSSolver(board)

# print how long it took to solve the puzzle
print("BFS finished in {0:.2f} seconds".format(timer.elapsedTime()))

# if we found the goal
if solver.found:
    print(f"Found the solution after {solver.moves} moves")
    solver.print_solution() # uncomment this line to see the solution
else:
    print("No solution found")

BFS finished in 0.14 seconds
Found the solution after 11 moves
Number of moves = 11
 1  0  2 
 7  5  4 
 8  6  3 

 1  5  2 
 7  0  4 
 8  6  3 

 1  5  2 
 7  4  0 
 8  6  3 

 1  5  2 
 7  4  3 
 8  6  0 

 1  5  2 
 7  4  3 
 8  0  6 

 1  5  2 
 7  4  3 
 0  8  6 

 1  5  2 
 0  4  3 
 7  8  6 

 1  5  2 
 4  0  3 
 7  8  6 

 1  0  2 
 4  5  3 
 7  8  6 

 1  2  0 
 4  5  3 
 7  8  6 

 1  2  3 
 4  5  0 
 7  8  6 

 1  2  3 
 4  5  6 
 7  8  0 



Run the `BFSSolver` on `puzzle2x2-01.txt`, `puzzle3x3-01.txt`, and `puzzle3x3-02.txt`. How long it takes solve each board in seconds? What is the number of moves for each case?

In [8]:
# Fill in the following variables with time (in seconds) for solving each of the boards. (float)
bfs_time_puzzle_2_2_1 = 0.00       # time for puzzle2x2-01.txt
bfs_time_puzzle_3_3_1 = 0.02       # time for puzzle3x3-01.txt
bfs_time_puzzle_3_3_2 = 0.13       # time for puzzle3x3-02.txt

# Fill in the number of moves taking to solve each of the boards, (int)
bfs_moves_puzzle_2_2_1 = 1       # moves for puzzle2x2-01.txt
bfs_moves_puzzle_3_3_1 = 8       # moves for puzzle3x3-01.txt
bfs_moves_puzzle_3_3_2 = 11       # moves for puzzle3x3-02.txt

### END CODE HERE ###

#### Activity 6 <a class="anchor" id="act6"></a> &nbsp; [Top](#top)

The file `dls_solver.py` has an implementation of the depth-limited search (DLS) algorithm. Unfortunately, as we have learned in the lectures, DLS is *incomplete*. However, a simple extension to DLS is the depth-first iterative deepening (DFID) algorithm which repeatedly calls DLS until it finds a solution or runs out of search options. 

Using the implementation of the DLS algorithm, implement DFID by completing the code in `dfid_solver.py`. Ensure that your code follows the algorithm that we studied in the class. 

Remember to fill in the following comments at the top of the source code file.

```Python
# FileName    : dfid_solver.py
# Student ID  : <YOUR ID>
# Student Name: <YOUR NAME>
```

#### The `DFIDSolver` Client Code

Client code for solving the 8-puzzle problem using the *Depth_First_Iterative_Deepening* approached.

In [9]:
from search.puzzle import Board      # used for the board configuration
from stdlib import StopwatchCPU      # a utility to compute the execution time in second
from search.puzzle import DFIDSolver  # the solver class

# create a board by reading from a file
board = Board(filename="data/puzzle/puzzle3x3-02.txt")

# create timer object
timer = StopwatchCPU()

# create a solver
solver = DFIDSolver(board)

# print how long it took to solve the puzzle
print("DLS finished in {0:.2f} seconds".format(timer.elapsedTime()))

# if we found the goal
if solver.found:
    print(f"Found the solution after {solver.moves} moves")
    solver.print_solution() # uncomment this line to see the solution
else:
    print("No solution found")

DLS finished in 0.37 seconds
Found the solution after 11 moves
Depth = 13
Number of moves = 11
 1  0  2 
 7  5  4 
 8  6  3 

 1  5  2 
 7  0  4 
 8  6  3 

 1  5  2 
 7  4  0 
 8  6  3 

 1  5  2 
 7  4  3 
 8  6  0 

 1  5  2 
 7  4  3 
 8  0  6 

 1  5  2 
 7  4  3 
 0  8  6 

 1  5  2 
 0  4  3 
 7  8  6 

 1  5  2 
 4  0  3 
 7  8  6 

 1  0  2 
 4  5  3 
 7  8  6 

 1  2  0 
 4  5  3 
 7  8  6 

 1  2  3 
 4  5  0 
 7  8  6 

 1  2  3 
 4  5  6 
 7  8  0 



Run the `DFIDSolver` on `puzzle2x2-01.txt`, `puzzle3x3-01.txt`, and `puzzle3x3-02.txt`. How long it takes solve each board in seconds? What is the number of moves for each case? At what depth does DFID find a solution to each one of these boards?

In [10]:
# Fill in the following variables with time (in seconds) for solving each of the boards. (float)
dfid_time_puzzle_2_2_1 = 0.00       # time for puzzle2x2-01.txt
dfid_time_puzzle_3_3_1 = 0.02       # time for puzzle3x3-01.txt
dfid_time_puzzle_3_3_2 = 0.36       # time for puzzle3x3-02.txt

# Fill in the number of moves taking to solve each of the boards, (int)
dfid_moves_puzzle_2_2_1 = 1        # moves for puzzle2x2-01.txt
dfid_moves_puzzle_3_3_1 = 8        # moves for puzzle3x3-01.txt
dfid_moves_puzzle_3_3_2 = 11       # moves for puzzle3x3-02.txt

# Fill in the following variables depth found by DLS to solve the boards. (int)
dfid_depth_puzzle_2_2_1 = 1        # depth for puzzle2x2-01.txt
dfid_depth_puzzle_3_3_1 = 8        # depth for puzzle3x3-01.txt
dfid_depth_puzzle_3_3_2 = 13       # depth for puzzle3x3-02.txt

### END CODE HERE ###

#### Activity 7 <a class="anchor" id="act7"></a> &nbsp; [Top](#top)

Based on the implementations of `BFSSolver`, `DFSSolver`, and `DFIDSolver`, answer all the following questions. 

* __Q1__ What is the minimum number of moves required for solving the puzzle in `puzzle3x3-02.txt`?

In [11]:
### START CODE HERE  (Activity 7: Q1) ###

# minimum number of moves required for solving the puzzle `puzzle3x3-02.txt`. (int)
min_puzzle_3_3_2 = 11       # minimum moves to solve  puzzle3x3-02.txt

### END CODE HERE ###

* __Q2__ How many moves does it take **DFS** to solve`puzzle3x3-02.txt`? How many moves doe **BFS** and **DFID** take for the same puzzle?

In [12]:
### START CODE HERE  (Activity 7: Q2) ###

# moves to solve puzzle3x3-02 using DFS, BFS and DFID (int)
moves_puzzle_3_3_2_dfs  = 90195         # DFS  moves to solve puzzle3x3-02.txt
moves_puzzle_3_3_2_bfs  = 11            # BFS  moves to solve puzzle3x3-02.txt
moves_puzzle_3_3_2_dfid = 11            # DFID moves to solve puzzle3x3-02.txt

### END CODE HERE ###

* __Q3__ Which algorithms is better in terms of the time taken to solve a puzzle, **DFS**, **BFS**, or **DFID**? Explain your answer based on the results of running these algorithm on both `puzzle3x3-01.txt` and `puzzle3x3-02.txt`. Write your answer as triple triple-quoted string at the cell below.

In [13]:
### START CODE HERE  (Activity 7: Q2) ###

answer = """
based on running these algorithm `puzzle3x3-01.txt` and `puzzle3x3-02.txt` 
the results of these algorithm as following:

- DFS solver took's 989.44 sec to solve puzzle3x3-01 and 2560.66 sec to solve puzzle3x3-02

- BFS solver took's 0.02 sec to solve puzzle3x3-01 and 0.13 sec to solve puzzle3x3-02

- DFID solver took's 0.02 sec to solve puzzle3x3-01 and 0.36 sec to solve puzzle3x3-02

So, I can conclude that BFS is the better algorithm in terms of the time token 
to solve these puzzle(`puzzle3x3-01.txt` and `puzzle3x3-02.txt`), also in terms of optimality and completely.
"""

### END CODE HERE ###

## 5 Code for the Path Finding Problem
Path finding problems are common and they are manifested in a variety of applications. Some, such as Web sites and in-car navigation systems can be defined in terms of locations and transitions along link among them. Others, such as video streaming in computer networks and airline travel planning require more complex specifications. 

In the code that comes with this lab you have implementations of *BFS*, *DLS* and *DFID* for finding the path between two nodes in an undirected graph. You have been also provided an **erroneous implementation of DFS** which you will have to fix in the following activities.

### 5.1 Activities on the Path Finding Problem <a class="anchor" id="path"></a>

#### Activity 8 <a class="anchor" id="act8"></a> &nbsp; [Top](#top)
Explore the code and input files that associated with this lab and answer each of the following. questions

* What is the name of the package that contains the path-finding code?
* Study the input files `tinyG.txt` and `mediumG.txt`? How many vertices and edges in graphs defined by these files?
* Draw the graph that corresponds to `tinyG.txt`. Can you identify any cycles in this graph. Write down at least one cycle in the cell below. 

In [14]:
### START CODE HERE  (Activity 8) ###

# Fill in the name of the package the contains the path-finding code (str)
path_package = 'search.graph'


# fill in number vertices and edges graphs defined in tinyG.txt and mediumG.txt (int)
tiny_g_vertices   = 13        # number of vertices in tinyG.txt
tiny_g_edges      = 13        # number of edges in tinyG.txt  
medium_g_vertices = 250       # number of vertices in mediumG.txt
medium_g_edges    = 1273      # number of edges in mediumG.txt


# write down one cycle in tinyG.txt using the format (v1->v2->v3->v1) (str)
tiny_g_cycle      = '(v5 -> v3 -> v4 -> v5)'      # a cycle in the graph defined by tinyG.txt 

### END CODE HERE ###

#### The `BFSPath` Client Code

Study the code provided to you in `bfs_path.py`, `dls_path.py` and `dfid_path.py`. Next run the following client code which uses the `BFSPath` class on `tinyG.txt` and `mediumG.txt`. Try different source and target vertices. Study the outout of the code carefully.

Using `BFSPath` class on `tinyG.txt`

In [15]:
# import the necessary libraries
from stdlib import Graph
from search.graph import BFSPath

# read a graph from a file
graph = Graph(filename="data/graph/tinyG.txt")

# try to find a path in graph from 0 to 4
# path = BFSPath(graph, 0, 4)

# try to find a path in graph from 4 to 0
# path = BFSPath(graph, 4, 0)

# try to find a path in graph from 7 to 0
# path = BFSPath(graph, 0, 7)

# try to find a path in graph from 12 to 10
path = BFSPath(graph, 12, 10)

# print the result 
if path.found:
    path.print_solution() # print the path
else:
    print("No path found")

Path: 12 -> 9 -> 10


`DLSPath` class on `tinyG.txt`

In [16]:
# import the necessary libraries
from stdlib import Graph
from search.graph import DLSPath

# read a graph from a file
graph = Graph(filename="data/graph/tinyG.txt")

# try to find a path in graph from 0 to 4, and 2 limit
# path = DLSPath(graph, 0, 4, 2)

# try to find a path in graph from 4 to 0, and 2 limit
# path = DLSPath(graph, 4, 0, 2)

# try to find a path in graph from 4 to 0, and 1 limit
# path = DLSPath(graph, 4, 0, 1)

# try to find a path in graph from 7 to 0, and 1 limit
# path = DLSPath(graph, 0, 7, 1)

# try to find a path in graph from 12 to 10, and 3 limit
path = DLSPath(graph, 12, 10, 3)

# print the result
if path.found:
    path.print_solution() # print the path
else:
    print("No path found")

Path: 12 -> 11 -> 9 -> 10


`DFIDPath` class on `tinyG.txt`

In [17]:
# import the necessary libraries
from stdlib import Graph
from search.graph import DFIDPath

# read a graph from a file
graph = Graph(filename="data/graph/tinyG.txt")

# try to find a path in graph from 0 to 4
# path = DFIDPath(graph, 0, 4)

# try to find a path in graph from 4 to 0
# path = DFIDPath(graph, 4, 0)

# try to find a path in graph from 7 to 0
# path = DFIDPath(graph, 0, 7)

# try to find a path in graph from 12 to 10
path = DFIDPath(graph, 12, 10)

# print the result
if path.found:
    path.print_solution() # print the path
    print(f"At depth: {path.depth}") # print the depth
else:
    print("No path found")

Path: 12 -> 11 -> 9 -> 10
At depth: 3


#### Activity 9 <a class="anchor" id="act9"></a> &nbsp; [Top](#top)
 
Using the client code of `BFSPath` as a guide, write the code that uses the implementation of *DFID algorithm* to find the path between any two vertices in the graph defined by `mediumG.txt`.

Using `DFIDPath` class on `mediumG.txt`

In [20]:
### START CODE HERE  (Activity 9) ###

# import the necessary libraries (two lines)
from stdlib import Graph
from search.graph import DFIDPath

# read a graph from a file (one line)
graph = Graph(filename="data/graph/mediumG.txt")

# create a DFID path object (one line)
# try to find a path in graph from 0 to 4
# path = DFIDPath(graph, 0, 4)

# try to find a path in graph from 4 to 0
# path = DFIDPath(graph, 4, 0)

# try to find a path in graph from 0 to 249
# path = DFIDPath(graph, 0, 249)

# try to find a path in graph from 1 to 0
path = DFIDPath(graph, 1, 0)

# print the result (4 lines)
if path.found:
    path.print_solution() # print the path
    print(f"At depth: {path.depth}") # print the depth
else:
    print("No path found")

### END CODE HERE ###

Path: 1 -> 220 -> 20 -> 247 -> 40 -> 194 -> 61 -> 234 -> 87 -> 136 -> 55 -> 4 -> 240 -> 26 -> 5 -> 226 -> 32 -> 248 -> 44 -> 0
At depth: 19


### .other client code

Using `BFSPath` class on `mediumG.txt`

In [18]:
# import the necessary libraries
from stdlib import Graph
from search.graph import BFSPath

# read a graph from a file
graph = Graph(filename="data/graph/mediumG.txt")

# try to find a path in graph from 0 to 4
# path = BFSPath(graph, 0, 4)

# try to find a path in graph from 4 to 0
# path = BFSPath(graph, 4, 0)

# try to find a path in graph from 0 to 249
# path = BFSPath(graph, 0, 249)

# try to find a path in graph from 1 to 0
path = BFSPath(graph, 1, 0)

# print the result
if path.found:
    path.print_solution() # print the path
else:
    print("No path found")

Path: 1 -> 107 -> 173 -> 128 -> 78 -> 77 -> 187 -> 160 -> 0


Using `DLSPath` class on `mediumG.txt`

In [19]:
# import the necessary libraries
from stdlib import Graph
from search.graph import DLSPath

# read a graph from a file
graph = Graph(filename="data/graph/mediumG.txt")

# try to find a path in graph from 0 to 4, and 8 limit
# path = DLSPath(graph, 0, 4, 8)

# try to find a path in graph from 4 to 0, and 2 limit
# path = DLSPath(graph, 4, 0, 2)

# try to find a path in graph from 0 to 249, and 22 limit
# path = DLSPath(graph, 0, 249, 22)

# try to find a path in graph from 1 to 0, and 19 limit
path = DLSPath(graph, 1, 0, 19)

# print the result
if path.found:
    path.print_solution() # print the path
else:
    print("No path found")

Path: 1 -> 220 -> 20 -> 247 -> 40 -> 194 -> 61 -> 234 -> 87 -> 136 -> 55 -> 4 -> 240 -> 26 -> 5 -> 226 -> 32 -> 248 -> 44 -> 0


#### Activity 10 <a class="anchor" id="act10"></a> &nbsp; [Top](#top)

The class `DFSPath` provides an implementation for finding a path between two nodes in a graph. The method is called `__dfs()`, and we list its code below.

```
1:      def __dfs(self, graph:Graph, source:int, goal:int):
2:          """
3:          Run DFS on a graph to find a path from source to goal
4:          """
5:  
6:          agenda  = Stack()
7:  
8:          firstNode = Node(source, 0, 0, None)
9:          agenda.push(firstNode)
10:  
11:          # DFS Loop
12:          while not(agenda.isEmpty()):
13:              node = agenda.pop()
14:  
15:              if node.state == goal:
16:                  self._goal = node # remeber the goal node
17:                  self._found = True
18:                  break # exist the loop
19:              
20:              # put the successor nodes in the stack.
21:              for successor in graph.adj(node.state):
22:  
23:                  # ctreate a sucessor node (sN)
24:                  sN = Node(successor, node.depth + 1, node.cost + 1, node)
25:                  agenda.push(sN)
```

Suppose you run `DFSPath` on any (*undirected*) graph. In that case, likely, it will _**never**_ return a solution (i. e., a path) because the given implementation of the DFS algorithm will run into an *infinite* loop. _**Your goal is to figure out what is missing in the code and add it. Next, write the client code for DFSPath in the cell below to test your code.**_

In [21]:
### START CODE HERE  (Activity 10) ###

# import the necessary libraries (two lines)
from stdlib import Graph
from search.graph import DFSPath

# read a graph from a file (one line)
# graph = Graph(filename="data/graph/mediumG.txt")
graph = Graph(filename="data/graph/tinyG.txt")

# create a DFS path object (one line)
# try to find a path in graph from 0 to 4
path = DFSPath(graph, 0, 4)

# try to find a path in graph from 4 to 0
# path = DFSPath(graph, 4, 0)

# try to find a path in graph from 7 to 0
# path = DFSPath(graph, 0, 7)

# try to find a path in graph from 12 to 10
# path = DFSPath(graph, 12, 10)

# print the result (4 lines)
if path.found:
    path.print_solution() # print the path
else:
    print("No path found")

### END CODE HERE ###

Path: 0 -> 6 -> 4


## Acknowledgement

Preparation of this lab material would not have been possible without  the adaptation code and content from [Sedgwick and Wayne (2011)](#ref1).  The author gratefully acknowledges the work of the authors cited while assuming complete responsibility for any mistake introduced in the adaptation.

## References

* <a class="anchor" id="ref1"></a>Sedgewick, R. and Wayne, K. (2011). Algorithms, 4th Edition. Addison-Wesley